# Project Notebook: Optimizing DataFrames and Processing in Chunks

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [ ]:
# Importing pandas
# import pandas as pd
pd.options.display.max_columns = 99


import pandas as pd

# Read in the first five lines of the loans_2007.csv file
df = pd.read_csv("https://bit.ly/3H2XVgC")
nrows=5


# Check for data quality issues
df.info()
df.describe()
df.head()



In [ ]:
import pandas as pd

# Read in the first 1000 rows of the loans_2007.csv file
df = pd.read_csv("https://bit.ly/3H2XVgC")
nrows=1000

# Calculate the memory usage of the dataframe
df_memory_usage = df.memory_usage().sum() / 1024**2
print(f"Total memory usage: {df_memory_usage:.2f} MB")


## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

In [ ]:
# Select only the columns with a numeric data type
numeric_columns = df.select_dtypes(include=["int", "float"])

# Count the number of columns
num_numeric_columns = numeric_columns.size

# Print the number of numeric columns
print(num_numeric_columns)



In [ ]:
# Select only the columns with a string data type
string_columns = df.select_dtypes(include=["object"])

# Count the number of columns
num_string_columns = string_columns.size

# Print the number of string columns
print(num_string_columns)



In [ ]:
# Select only the columns with a string data type
string_columns = df.select_dtypes(include=["object"])

# Iterate through the string columns
for column in string_columns:
    # Get the number of unique values
    num_unique = df[column].nunique()
    
    # Get the total number of values
    num_total = df[column].size
    
    # Calculate the percentage of unique values
    pct_unique = num_unique / num_total
    
    # Print the column name and percentage of unique values
    print(f"Column {column}: {pct_unique:.2%} unique")



In [ ]:
import pandas as pd

# Initialize a list to store the names of the float columns with no missing values
members = []

# Iterate over the dataframe chunks
for chunk in pd.read_csv("https://bit.ly/3H2XVgC", chunksize=1000):
  # Select only the float columns
  float_columns = chunk.select_dtypes(include='float')

  # Check if the float columns have any missing values
  no_missing_values = float_columns.isnull().sum().eq(0)

  # Get the names of the float columns with no missing values
  chunk_members = no_missing_values[no_missing_values.all()].index

  # Add the float columns with no missing values to the list
  members.extend(chunk_members)

print(members)





In [16]:
import pandas as pd

# Initialize a variable to store the total memory usage
total_memory_usage = 0

# Iterate over the dataframe chunks
for chunk in pd.read_csv("https://bit.ly/3H2XVgC" , chunksize=1000):
  # Calculate the memory usage of each column
  column_memory_usage = chunk.memory_usage()

  # Add the total memory usage of the chunk to the total
  total_memory_usage += column_memory_usage.sum()

print(total_memory_usage)



17701480


## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [ ]:
# Your code goes here
import pandas as pd

# Read in the dataframe chunk
df = pd.read_csv("https://bit.ly/3H2XVgC")

# Remove the % sign from the int_rate column and convert it to a float
df["int_rate"] = df["int_rate"].str.rstrip("%").astype(float)

# Convert the grade and sub_grade columns to the category type
df["grade"] = df["grade"].astype("category")
df["sub_grade"] = df["sub_grade"].astype("category")

# Calculate the total memory footprint of the dataframe
total_memory = df.memory_usage(deep=True).sum()
print("Total memory usage:", total_memory)


## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




In [ ]:
# Your code goes here
# Select float columns
float_cols = df.select_dtypes(include='float').columns

# Select float columns with missing values
float_cols_with_missing = df[float_cols].isnull().sum()[df[float_cols].isnull().sum() > 0]

# Print float columns with missing values
print(float_cols_with_missing)



In [ ]:
# Select float columns
float_cols = df.select_dtypes(include='float').columns

# Select float columns with no missing values
float_cols_no_missing = df[float_cols].isnull().sum()[df[float_cols].isnull().sum() == 0]

# Print float columns with no missing values
print(float_cols_no_missing)

# Convert float columns to integers



# Iterate over the chunks
for chunk in pd.read_csv("https://bit.ly/3H2XVgC" , chunksize=chunksize):
    # Select float columns
    float_cols = chunk.select_dtypes(include='float').columns
    
    # Convert float columns with missing values to float32
    chunk[float_cols] = chunk[float_cols].astype('float32')
    
    # Select float columns with no missing values
    float_cols_no_missing = chunk[float_cols].isnull().sum()[chunk[float_cols].isnull().sum() == 0]
    
    # Convert float columns with no missing values to integers
    chunk[float_cols_no_missing.index] = pd.to_numeric(chunk[float_cols_no_missing.index], downcast='integer')
    
    # Append chunk to list
    chunks.append(chunk)

# Concatenate the chunks and save to a new DataFrame
df_optimized = pd.concat(chunks)



In [ ]:
#
# Calculate the total memory usage of the DataFrame
total_memory_usage = df.memory_usage(deep=True).sum()

# Calculate the total memory usage of the optimized DataFrame
optimized_memory_usage = df_optimized.memory_usage(deep=True).sum()

# Print the memory usage of the DataFrame and the optimized DataFrame
print("Total memory usage:", total_memory_usage)
print("Optimized memory usage:", optimized_memory_usage)

# Calculate the difference in memory usage
memory_saving = total_memory_usage - optimized_memory_usage

# Print the difference in memory usage
print("Memory saving:", memory_saving)


## Next Steps

We've practiced optimizing a dataframe's memory footprint and working with dataframe chunks. Here's an idea for some next steps:

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

* Determine the optimal chunk size based on the memory constraints you provide.

* Determine which string columns can be converted to numeric ones by removing the `%` character.

* Determine which numeric columns can be converted to more space efficient representations.


In [ ]:
import pandas as pd


def get_optimal_chunk_size(df, memory_limit):
  # Calculate the size of the dataframe
  df_size = df.memory_usage(deep=True).sum() / (1024 ** 2)

  # Calculate the optimal chunk size
  chunk_size = int(memory_limit / df_size * len(df))

  return chunk_size

  memory_limit = 1024 # Set the memory limit to 1GB

df = pd.read_csv("https://bit.ly/3H2XVgC") # Read in the dataframe

chunk_size = get_optimal_chunk_size(df, memory_limit)




In [11]:
def get_numeric_columns(df):
  numeric_columns = []
  for col in df.select_dtypes(include=['object']).columns:
    try:
      df[col] = pd.to_numeric(df[col].str.strip().str.replace('%', ''))
      numeric_columns.append(col)
    except ValueError:
      pass
  return numeric_columns

  df = pd.read_csv("https://bit.ly/3H2XVgC") # Read in the dataframe
numeric_columns = get_numeric_columns(df)


In [12]:
# Determine which numeric columns can be converted to more space efficient representations.

def get_space_efficient_columns(df):
  space_efficient_columns = []
  for col in df.select_dtypes(include=['float']).columns:
    df[col] = pd.to_numeric(df[col], downcast='float')
    space_efficient_columns.append(col)
  for col in df.select_dtypes(include=['int']).columns:
    df[col] = pd.to_numeric(df[col], downcast='integer')
    space_efficient_columns.append(col)
  return space_efficient_columns

df = pd.read_csv("https://bit.ly/3H2XVgC") # Read in the dataframe
space_efficient_columns = get_space_efficient_columns(df)
